In [48]:
from random import shuffle, uniform, randint
import math
class GeneticTSP:
    def __init__(self, graph_matrix, elitism=True, tournament_size=3, mutation_rate=0.5, num_generations=1000, population_size=200):
        self.graph_matrix = graph_matrix
        self.genome_length = len(graph_matrix)
        self.mutation_rate = mutation_rate
        self.elitism = elitism
        self.tournament_size = tournament_size
        self.num_generations = num_generations
        self.population_size = population_size
        self.current_best_solution = {
            "tour": [],
            "tour_length": math.inf
        }
        self.run_data = []

    def _update_best_solution(self, tour, tour_length):
        self.current_best_solution["tour"] = tour
        self.current_best_solution["tour_length"] = tour_length

    def update_run_data(self):
        self.run_data.append(self.current_best_solution)


    def create_individual(self):
        individual = [i for i in range(self.genome_length)]
        shuffle(individual)
        return individual

    def evaluate_individual(self, indiviudal):
        accum_distance = 0
        for i in range(self.genome_length-1):
            a = indiviudal[i]
            b = indiviudal[i+1]
            accum_distance += self.graph_matrix[a][b]
        last_city = indiviudal[-1]
        origin = indiviudal[0]
        accum_distance += self.graph_matrix[last_city][origin]
        return accum_distance

    def create_population(self, population_size):
        population = []
        for _ in range(population_size):
            new_individual = self.create_individual()
            population.append(new_individual)
        return population

    def get_fittest(self, population, minimize=True):
        population_size = len(population)
        fittest = 0
        for i in range(population_size):
            score = self.evaluate_individual(population[i])
            if score <= self.evaluate_individual(population[fittest]):
                if minimize:
                    fittest = i
            else:
                if not minimize:
                    fittest = i
        return population[fittest]

    def mutate(self, individual):
        for i in range(len(individual)):
            if uniform(0, 1) < self.mutation_rate:
                swap_index = randint(0, len(individual) - 1)
                temp = individual[i]
                individual[i] = individual[swap_index]
                individual[swap_index] = temp
        return individual

    def crossover(self, individual1, individual2):
        new_individual = [0 for i in range(len(individual1))]
        indx_1 = randint(0, len(individual1) - 1)
        indx_2 = randint(0, len(individual1) - 1)
        start_pos = min(indx_1, indx_2)
        end_pos = max(indx_1, indx_2)
        temp = individual1[start_pos:(end_pos+1)]
        new_individual[start_pos:(end_pos+1)] = temp
        pos = 0
        for i in range(len(new_individual)):
            while start_pos <= pos <= end_pos:
                pos += 1
            if individual2[i] not in temp:
                new_individual[pos] = individual2[i]
                pos += 1
        return new_individual

    def evolve(self, population):
        new_population = []
        elite_offset = 0
        if self.elitism:
            elite = self.get_fittest(population)
            new_population.insert(0, elite)
            elite_offset = 1
        for i in range(elite_offset, len(population)):
            parent1 = self.select_candidate(population)
            parent2 = self.select_candidate(population)
            new_individual = self.crossover(parent1, parent2)
            new_population.insert(i, new_individual)

        for i in range(elite_offset, len(population)):
            mutated_individual = self.mutate(new_population[i])
            new_population[i] = mutated_individual
        return new_population

    def select_candidate(self, population):
        tournament = []
        for _ in range(self.tournament_size):
            rand_index = randint(0, len(population) - 1)
            tournament.append(population[rand_index])
        fittest = self.get_fittest(tournament)
        return fittest

    def run(self):
        population = self.create_population(self.population_size)
        for _ in range(self.num_generations):
            population = self.evolve(population)
            fittest = self.get_fittest(population)
            score = self.evaluate_individual(fittest)
            self._update_best_solution(fittest, score)
            if _ % 50 == 0: 
                print(_, self.current_best_solution)
            self.update_run_data()
            self.mutation_rate = self.mutation_rate*0.9
            if self.mutation_rate < 0.005:
                self.mutation_rate = 0.05

        return self.run_data

In [49]:
import random
import math
import pandas as pd
# pip install xlrd
import networkx as nx
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

full_path = "testdata.xlsx"
# Load a dataset
df = pd.read_excel(full_path, sheet_name="ATT48", header=None)
graph = df.values

# Genetic Algorithm
print("Running Genetic Algorithm")
run_data = GeneticTSP(graph).run()


Running Genetic Algorithm
0 {'tour': [39, 24, 20, 47, 46, 42, 13, 40, 45, 15, 23, 44, 9, 1, 7, 6, 18, 17, 33, 31, 11, 14, 27, 26, 10, 37, 29, 30, 16, 43, 8, 35, 21, 19, 36, 22, 28, 25, 32, 0, 2, 34, 5, 12, 38, 4, 3, 41], 'tour_length': 124044}
50 {'tour': [47, 33, 32, 39, 11, 45, 27, 12, 14, 30, 42, 26, 22, 46, 2, 0, 40, 21, 15, 7, 37, 5, 43, 19, 29, 18, 36, 16, 35, 10, 23, 1, 25, 34, 44, 41, 4, 24, 13, 38, 28, 9, 3, 8, 17, 6, 31, 20], 'tour_length': 81449}
100 {'tour': [47, 38, 12, 22, 46, 2, 33, 0, 40, 15, 21, 13, 31, 10, 7, 37, 8, 30, 45, 35, 32, 42, 16, 6, 36, 26, 43, 14, 27, 17, 5, 29, 18, 19, 11, 39, 24, 20, 4, 28, 1, 25, 34, 44, 23, 41, 9, 3], 'tour_length': 60863}
150 {'tour': [2, 33, 40, 15, 7, 8, 0, 21, 22, 13, 47, 31, 38, 10, 12, 20, 11, 37, 30, 45, 32, 29, 16, 42, 26, 43, 17, 19, 14, 35, 5, 27, 6, 36, 18, 46, 39, 24, 4, 28, 1, 25, 3, 34, 44, 23, 9, 41], 'tour_length': 53078}
200 {'tour': [47, 13, 33, 40, 15, 7, 0, 21, 2, 46, 12, 31, 38, 10, 11, 39, 32, 8, 37, 30, 45, 16, 29